In [1]:
import nltk
import re
from nltk.corpus import stopwords
import pandas as pd
import timeit
import numpy as np
import sys
from nltk.sentiment.vader import SentimentIntensityAnalyzer

/Users/nathan/.pyenv/versions/3.5.2/envs/stats/lib/python3.5/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [41]:
# Process the amazon review text



def review_to_words(review_text, stops):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    # 1. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 2. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 3. Remove stop words
    meaningful_words = [w for w in words if not w in stops and len(w)>3]   
    #
    # 4. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words ))


#  In Python, searching a set is much faster than searching
#  a list, so convert the stop words to a set
stops = set(stopwords.words("english"))                  






In [6]:
# Initialize key variables

path = "/Users/nathan/Documents/Stats/SignalDataScience/finalproject/datafiles/"
target = "chunk1.csv"
destination_file = "processed1.csv"
debug = True

In [47]:
# load df and add new col for processed reviews
df = pd.read_json(path + target)
print(list(df))
df["sentiment"] = np.nan

['asin', 'helpful', 'overall', 'reviewText', 'reviewTime', 'reviewerID', 'reviewerName', 'summary', 'unixReviewTime']


In [51]:
print(list(df))
print(df.shape[0])


['asin', 'helpful', 'overall', 'reviewText', 'reviewTime', 'reviewerID', 'reviewerName', 'summary', 'unixReviewTime', 'pros_text', 'sentiment']
50001


AttributeError: module 'nltk' has no attribute 'version'

In [74]:
def df_sentiment(df):
    stops = set(stopwords.words("english"))  
    # if debug == True, only try for try_minutes
    try_minutes = 3
    count = 0
    start_time = int(timeit.default_timer())
    old_count = 1
    sid = SentimentIntensityAnalyzer()
    df['vader'] = np.nan
    for row in range(df.shape[0]):
        if df.ix[row, "vader"] == np.nan:
            # df.set_value(row,"pros_text",(review_to_words(df.ix[row, "reviewText"], stops)))
            sentiment_list = 0
            lines_list = nltk.tokenize.sent_tokenize(df.ix[row, "reviewText"])
            for sentence in lines_list:
                ss = sid.polarity_scores(sentence)
                sentiment_list += ss['compound']
            df.set_value(row,"vader",sentiment_list)
        count += 1
        time_e = (int(timeit.default_timer()) - start_time)/60
        if time_e >= try_minutes:
            # df.to_csv(path + destination_file)
            # print(df.head())
            print("Out of time. Total reviews sentiment processed: ", count)
            return df
        if count % 10000 == 0:
            if old_count > 1: print("", end = "\r")
            print("Reviews sentiment processed: ", count, end = "")
            old_count = count
            # sys.stdout.flush()
    return df

Reviews processed:  46700

TypeError: 'method' object is not subscriptable

In [46]:
print(df.head())

            asin helpful  overall  \
0     0000000078  [1, 1]        5   
1     0000000116  [5, 5]        4   
10    0000013714  [0, 0]        5   
100   0000031887  [0, 0]        5   
1000  000107461X  [0, 0]        5   

                                             reviewText   reviewTime  \
0     Conversations with God Book 1 is the single mo...  08 11, 2004   
1     Interesting Grisham tale of a lawyer that take...  04 27, 2002   
10    This is a large size hymn book which is great ...   03 9, 2013   
100   I wanted it for a one time use, and after that...   04 8, 2014   
1000  the music was everything that I wanted. Would ...   02 3, 2013   

          reviewerID    reviewerName                    summary  \
0     A3AF8FFZAZYNE5            None                 Impactful!   
1      AH2L9G3DQHHAJ           chris         Show me the money!   
10     APOZ15IEYQRRR       maewest64          Awesome Hymn Book   
100   A1716964DOSFDG               D  You get what you pay for!   
1000  A1Z